In [2]:
import pandas as pd
from thefuzz import fuzz, process

In [3]:
# Đọc danh sách các country theo tên chuẩn lấy từ GeoNames
countries = pd.read_csv('../data/country.csv')["Country"]

In [4]:
# Đọc các tập dữ liệu
co2_df = pd.read_csv('../data/co2.csv').rename(columns={"value": "co2"})
climate_change_df = pd.read_csv('../data/climate_change.csv').rename(columns={"value": "climate_change"})
energy_df = pd.read_csv('../data/energy.csv').rename(columns={"value": "energy"})

In [5]:
def find_best_match(name):
  best_match = process.extractOne(name, countries, scorer=fuzz.WRatio)
  return best_match[0], best_match[1]

In [6]:
co2_df_country = co2_df.loc[:, ["country"]].drop_duplicates(subset="country")

In [7]:
matches = co2_df_country["country"].apply(find_best_match)
co2_df_country[["normalized_country", 'match_score']] = pd.DataFrame(matches.to_list(), index=co2_df_country.index)

In [8]:
co2_df_country[co2_df_country["match_score"] < 100].sort_values(by="match_score")
# kiểm tra match_score truớc
# bỏ global, international
# với mấy cái điểm thấp thì mapping bằng tay, ví dụ USA -> United State

,country,normalized_country,match_score
12604,Côte d'Ivoire,Israel,60
60554,Global,Gibraltar,60
54800,Türkiye,Turkey,67
57266,USA,Saint Barthelemy,72
60006,International Shipping,Tonga,72
60280,International Aviation,Latvia,75
9590,Cape Verde,Cabo Verde,80
45210,Ryukyu Islands,South Georgia and the South Sandwich Islands,86
39182,State of Palestine,Democratic Republic of the Congo,86
46032,Sint Maarten (Dutch part),Sint Maarten,90


In [10]:
# ví dụ 
co2_df["country"] = co2_df["country"].apply(lambda x: "United State" if x == "USA" else x)